## Load required modules.

In [1]:
%matplotlib widget
# Import ncessary modules.
from sklearn.decomposition import PCA
import xlrd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import ipywidgets 
from ipywidgets import interact, IntSlider
import ipywidgets as widgets
from matplotlib import pyplot
from IPython.display import display, clear_output, Javascript
from matplotlib.colors import LinearSegmentedColormap
from sklearn.preprocessing import FunctionTransformer
from matplotlib.legend import DraggableLegend, Legend
import matplotlib

from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.pipeline import Pipeline
import gc
import psutil


import customtkinter as ctk
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter import *

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from tkinter import IntVar
from tkinter import Menu
import matplotlib.gridspec as gridspec
from tkinter import filedialog as fd, simpledialog

In [2]:
# window = ctk.CTk()

# #window
# window.title('MineralAI')
# window.geometry('1100x800')

# #widgets


# window.mainloop()

In [3]:
class main(ctk.CTk):
    
    def __init__(self):
        super().__init__()
        self.title("MineralAI")
        self.geometry("1600x1600")
        self.filtered_df = pd.DataFrame()
        self.create_widgets()

    
    def create_widgets(self):
        self.toolbox_frame = ctk.CTkFrame(self, height=20)
        self.toolbox_frame.grid(row=0, column=0, columnspan=4, sticky="ew", padx=5, pady=(5,0))
    
        self.selection_frame = ctk.CTkFrame(self)
        self.selection_frame.grid(row=1, column=0, rowspan=1, sticky="ns", pady=(5,0), padx=5)
        
        self.legend_frame = ctk.CTkFrame(self, width=250)
        #self.output_frame.pack_propagate(False)
        self.legend_frame.grid(row=1, column=3, sticky="ns", padx=5, pady=(5,0))
    
        self.box_frame = ctk.CTkFrame(self, width=150)
        self.box_frame.grid(row=2, column=0, rowspan=2, sticky="nsew", padx=5, pady=5)
    
        self.box_frame_sub = ctk.CTkFrame(self, width=150)
        self.box_frame_sub.grid(row=1, column=1, rowspan=3, sticky="ns", pady=5)
    
        self.bargraph_frame = ctk.CTkFrame(self)
        self.bargraph_frame.grid(row=1, column=2, rowspan=3, sticky="nsew", pady=5, padx=(5,0))
        
        self.output_frame = ctk.CTkFrame(self, width=250)
        self.output_frame.pack_propagate(False)
        self.output_frame.grid(row=2, column=3, sticky="nsew", padx=5, pady=5)
        
        self.grid_columnconfigure(2, weight=1)
        self.grid_rowconfigure(2, weight=1)

        self.select_litho_button = ctk.CTkButton(self.toolbox_frame, text="Open a File", command=self.load_data)
        self.select_litho_button.pack(side="left", padx = 5, pady = 3)

        self.output_text = tk.Text(self.output_frame, wrap="word")
        self.output_text.pack(fill="both", expand=True)

        
        menubar = Menu(self)
        self.filemenu = Menu(menubar, tearoff=0)
        self.filemenu.add_command(label="New Window", command=self.new_window)
        self.filemenu.add_command(label="Open File", command=self.load_data)
        self.save_pc_menu = tk.Menu(self.filemenu, tearoff=0)
        
        self.save_pc_menu.add_command(label="Save PC by sample excel", command=self.export_pc_by_sample)
        self.save_pc_menu.add_command(label="Save PC by element excel", command=self.export_pc_by_element)
        
        self.save_pc_menu.entryconfig("Save PC by sample excel", state=tk.DISABLED)
        self.save_pc_menu.entryconfig("Save PC by element excel", state=tk.DISABLED)
        
        self.filemenu.add_cascade(label="Save PC", menu=self.save_pc_menu)
        
        self.filemenu.add_separator()
        self.filemenu.add_command(label="Exit", command=self.destroy)
        menubar.add_cascade(label="File", menu=self.filemenu)
        
        helpmenu = Menu(menubar, tearoff=0)
        helpmenu.add_command(label="Help Index", command=None)
        helpmenu.add_command(label="About...", command=None)
        menubar.add_cascade(label="Help", menu=helpmenu)
        self.config(menu=menubar)

    def new_window(self):
        new_win = main()
        new_win.title("MineralAI Window")
        new_win.mainloop()

    def button_0(self):
    
        self.sheet_combobox = ctk.CTkComboBox(self.selection_frame, values=[], state="readonly")
        self.sheet_combobox.pack(padx=5, pady=5)
        self.sheet_combobox.set("Select a sheet")
        
        self.load_sheet_button = ctk.CTkButton(self.selection_frame, text="Load Sheet", command=self.load_selected_sheet)
        self.load_sheet_button.pack(pady=5)

        
    def create_lithobuttons(self):
        ctk.CTkLabel(self.selection_frame, text="Select Lithology:").pack(side="top", anchor = "w", padx=5, pady=5)
        self.lithology_listbox.pack(side="top", fill = "x", padx = 5, pady = (5,0))

        self.create_buttons()

    def create_buttons(self):
        self.current_button = None

        ctk.CTkLabel(self.selection_frame, text="Select PCA scaler:").pack(side="top", anchor = "w", padx=5, pady=(5,0))

        self.scaler_combo = ctk.CTkComboBox(self.selection_frame, values=["Standard Scaler", "Logarithmic Scaler"])
        self.scaler_combo.pack(side="top", fill = "x", padx = 5, pady = 5)
        self.scaler_combo.set("Standard Scaler")

        self.apply_button = ctk.CTkButton(self.selection_frame, text="Apply", command=lambda: self.on_button_click(self.apply_button, self.filter_dataframe))
        self.apply_button.pack(side="top", pady = (0,5))


    def on_button_click(self, button, command):
        if self.current_button is not None:
            self.current_button.configure(fg_color='#3B8ED0') 
        button.configure(fg_color="grey")
        self.current_button = button
        command()


    def clear_all(self):
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()
        for widget in self.box_frame.winfo_children():
            widget.destroy()
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()
        # for widget in self.output_frame.winfo_children():
        #     widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
        for widget in self.selection_frame.winfo_children():
            widget.destroy()
    def clear(self):
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()
        for widget in self.box_frame.winfo_children():
            widget.destroy()
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()
        for widget in self.output_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

    
    def load_data(self):
        try:
            #self.file_path = 'Geochemistry Results-AGG reduced variables.xlsx'
            self.file_path = fd.askopenfilename()
            if not self.file_path:
                self.output_text.insert("end", "No file selected\n")
                return

            self.clear_all()
            self.button_0()
            self.sheet_names = pd.ExcelFile(self.file_path).sheet_names
            self.sheet_combobox.configure(values=self.sheet_names)
            self.output_text.insert("end", "File loaded successfully. Please select a sheet.\n")

        except:
            self.output_text.insert("end", f"Error loading file: \n")

    def load_selected_sheet(self):
        try:
            selected_sheet = self.sheet_combobox.get()
            if selected_sheet not in self.sheet_names:
                self.output_text.insert("end", "Invalid sheet name\n")
                return
            self.sheet_name=selected_sheet
            self.df_0 = pd.read_excel(self.file_path, self.sheet_name)
            
            for widget in self.selection_frame.winfo_children():
                widget.destroy()
            self.process_sheet()
            
        except:
            self.output_text.insert("end", f"Error loading sheet: \n")

    def process_sheet(self):
        if "Lithology" in self.df_0.columns:
            self.lithology_listbox = tk.Listbox(self.selection_frame, selectmode=tk.MULTIPLE)
            lithologies = self.df_0['Lithology'].unique()
            for lithology in lithologies:
                self.lithology_listbox.insert(tk.END, lithology)
            self.create_lithobuttons()

        else:
            self.output_text.insert("end", "Lithology column not found in dataframe\n")
            self.create_buttons()

    
    def filter_dataframe(self):
        self.clear()
        if hasattr(self, 'lithologies_label') and self.lithologies_label is not None:
            self.lithologies_label.destroy()
            del self.lithologies_label

        self.output_text = tk.Text(self.output_frame, wrap="word")
        self.output_text.pack(fill="both", expand=True)

        try:
            selected_indices = self.lithology_listbox.curselection()
            selected_lithologies = [self.lithology_listbox.get(i) for i in selected_indices]
            self.filtered_df = self.df_0[self.df_0['Lithology'].isin(selected_lithologies)]
            
            if not selected_lithologies:
                self.lithologies_label = ctk.CTkLabel(self.selection_frame, text="No lithology selected:(", font=("Arial", 12))
                self.lithologies_label.pack(side="top", padx=5, pady=(5,10))
            else:
                lithologies_text = "\n".join(selected_lithologies)
                self.lithologies_label = ctk.CTkLabel(self.selection_frame, text=f"Selected Lithologies:\n{lithologies_text}", font=("Arial", 12))
                self.lithologies_label.pack(side="top", padx=5, pady=(5,10))
                             
        except:
            self.filtered_df = self.df_0
        
        filtered_columns = [col for col in self.filtered_df.columns if '_ppm' in col or '_pct' in col]

        df_filtered = self.filtered_df.replace('<', '', regex=True)
        df_filtered[filtered_columns] = df_filtered[filtered_columns].apply(pd.to_numeric, errors='coerce')
        self.cleaned_df = df_filtered.dropna(subset=filtered_columns)
        
        df_filtered = self.cleaned_df[filtered_columns]

        self.df = df_filtered
        self.update_output()
        self.print_pca()

    
    def update_output(self):
        self.output_text.delete(1.0, "end")
        self.output_text.insert("end", f"Filtered & Cleaned df:\n{self.df.head()}\n")
        self.output_text.insert("end", f"Filtered & Cleaned df:\n{self.df.shape}\n")

    
    def print_pca(self):
        try:
            self.pca_instance = PCA_class(self.df, self.scaler_combo)
            self.output_text.insert("end", f"PCA performed successfully. Shape of transformed data: {self.pca_instance.x.shape}\n")
            self.output_text.insert("end", f"PCA Components:\n{self.pca_instance.pca_df.head()}\n")
            self.color_function()
            self.shape_map()
            self.loading_graph()   
            
        except ValueError as e:
            self.output_text.insert("end", f"{e}\n")

    def color_function(self):
        try:
            self.dc = self.cleaned_df['Lithology']
            
            self.dc = self.dc.to_frame()
            self.dc = self.dc.rename(columns={self.dc.columns[0]: 'Lithology'})
            
            # color_list = dc['Lithology'].unique()
            # default_colors = ["lime", "blue", "cyan", "deeppink", "pink", "black", "black", "black", "grey", "lightgrey"]
            # color_map = {}
            # # Populate the dictionary with symbols for each shape
            # for i, color in enumerate(color_list):
            #     if i < len(default_colors):
            #         color_map[color] = default_colors[i]
            self.color_map = {
                'Calcitic marble': 'lime',
                'Dolomitic marble': 'blue',
                'Di-Tr dolomitic marble': 'cyan',
                'Carbonatite': 'deeppink',
                'Carbonatite-like': 'pink',
                'Syenite': 'black',
                'Altered Syenite': 'black',
                'Syenite-like': 'black',
                'Impure Siliciclastic': 'grey',
                'Pure Siliciclastic': 'lightgrey'
            }
            self.dc['Color'] = self.dc['Lithology'].map(self.color_map)
            self.output_text.insert("end", self.dc)
        except Exception as e:
            self.color_map = {}
            self.dc = pd.DataFrame()
            self.output_text.insert("end", f"No Lithology found: {e}\n")
        
        
    def shape_map(self):
        try:
            self.ds = self.cleaned_df['Shape']
            self.ds = self.ds.to_frame()
            self.ds = self.ds.rename(columns={self.ds.columns[0]: 'Shape'})
            
            # shape_list = ds['Shape'].unique()
            # default_symbols = ["^", "*", "o", "s", "D"]
            # color_map1 = {}
            # # Populate the dictionary with symbols for each shape
            # for i, shape in enumerate(shape_list):
            #     if i < len(default_symbols):
            #         color_map1[shape] = default_symbols[i]
            
            self.color_map1 = {
                'Marble Units': "^",
                'Altered Intrusion': "*",
                'Siliciclastic': "o",
                'Intrusion': "s",
                'Anamolous Rock': "D",
            }  
            self.ds["Shapes"] = self.ds["Shape"].map(self.color_map1)
            self.output_text.insert("end", self.ds)
        except Exception as e:
            self.color_map1 = {}
            self.ds = pd.DataFrame()
            self.output_text.insert("end", f"No Shape found: {e}\n")


            
    def loading_graph(self):
        sample_class(self.pca_instance, self.df, self.cleaned_df, self.bargraph_frame, self.box_frame, self.box_frame_sub, self.on_button_click, self.apply_button)
        self.loading_instance = loading_class(self.pca_instance, self.df, self.bargraph_frame, self.box_frame, self.box_frame_sub, self.on_button_click, self.apply_button)
        threed_class(self.pca_instance, self.df, self.bargraph_frame, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.color_map, self.color_map1, self.on_button_click, self.apply_button, self.legend_frame)
        twod_class(self.pca_instance, self.df, self.cleaned_df, self.bargraph_frame, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.color_map, self.color_map1, self.on_button_click, self.apply_button, self.legend_frame)
        Cluster3DPlotClass(self.df, self.df_0, self.file_path, self.sheet_name, self.bargraph_frame, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.on_button_click, self.apply_button, self.legend_frame)
        Cluster2DPlotClass(self.df, self.df_0, self.file_path, self.sheet_name, self.bargraph_frame, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.on_button_click, self.apply_button, self.legend_frame)
        drill_class(self.pca_instance, self.df_0, self.df, self.bargraph_frame, self.box_frame, self.box_frame_sub, self.on_button_click, self.apply_button)
        
        self.save_pc_menu.entryconfig("Save PC by sample excel", state=tk.NORMAL)
        self.save_pc_menu.entryconfig("Save PC by element excel", state=tk.NORMAL)
        
        
    def export_pc_by_sample(self):
        self.pca_df_scaled = self.pca_instance.pca_df_scaled
        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return

        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            self.df_0["Sample Description"].to_excel(excel_writer, sheet_name='Sheet1', index=False, header=True)
            self.pca_df_scaled.to_excel(excel_writer, sheet_name='Sheet1', startcol=1, index=False)

    
    def export_pc_by_element(self):
        self.loadings = self.loading_instance.loadings
        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return
        
        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            columns_df = pd.DataFrame(self.df.columns, columns=['Column Names'])
            columns_df.to_excel(excel_writer, sheet_name='Sheet1', index=False, header=True)
            self.loadings.to_excel(excel_writer, sheet_name='Sheet1', startcol=1, index=False)






class drill_class:
    try:
        def __init__(self, pca_instance, df_0, df, bargraph_frame, box_frame, box_frame_sub, on_button_click, apply_button):
            self.apply_button = apply_button
            self.on_button_click = on_button_click
            self.pca = pca_instance.pca
            self.pca_df_scaled = pca_instance.pca_df_scaled
            self.df_0 = df_0
            self.df = df
            self.bargraph_frame = bargraph_frame
            self.box_frame = box_frame
            self.box_frame_sub = box_frame_sub
            self.var = IntVar()
            
            self.df_0['Sample_ID_Clean'] = self.df_0.iloc[:, 0].apply(lambda x: x.replace('-', ''))
            self.df_0['Drill_Hole'] = self.df_0['Sample_ID_Clean'].apply(lambda x: x[:3]) 
            self.df_0['Depth'] = self.df_0.iloc[:, 2]
            
            self.drill_holes = sorted(self.df_0['Drill_Hole'].unique().tolist()) 
            
            self.Graph_PCA()
    
        def Graph_PCA(self):
            self.graph_bar_button = ctk.CTkButton(self.box_frame, text="Drill PC graph", command=lambda: self.on_button_click(self.graph_bar_button, self.Graph_PCA_sub))
            self.graph_bar_button.pack(side="top", padx=5, pady=(5, 0))
    
        def Graph_PCA_sub(self):
            for widget in self.box_frame_sub.winfo_children():
                widget.destroy()
    
            ttk.Label(self.box_frame_sub, text="Select Drill Hole:").pack(side="top", padx=5, pady=(5, 0))
            self.drill_hole_combo = ttk.Combobox(self.box_frame_sub, values=self.drill_holes)
            self.drill_hole_combo.current(0)
            self.drill_hole_combo.pack(side="top", padx=5, pady=(5, 0))
    
            self.apply_Graph_PCA = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.update_plots)
            self.apply_Graph_PCA.pack(side="top", padx=5, pady=5)
    
        def normalize(self, values, vmin=None, vmax=None):
            vmin = vmin if vmin is not None else np.min(values)
            vmax = vmax if vmin is not None else np.max(values)
            norm_values = (values - vmin) / (vmax - vmin)
            return norm_values
    
        def plot_pca_scatter(self, data, y_positions, ax):
            norm_values = self.normalize(data, vmin=-max(abs(data)), vmax=max(abs(data)))
            scatter_colors = plt.cm.coolwarm(norm_values)
            ax.scatter(data, y_positions, color=scatter_colors, s=30)  
            ax.set_yticks(y_positions)
            ax.set_yticklabels([]) 
            ax.invert_yaxis()
            ax.grid(True, linestyle='--', linewidth=0.5)
            ax.axvline(x=0, color='black', linewidth=0.5)
            max_val = max(abs(data.min()), data.max())
            ax.set_xlim(-max_val, max_val)
            ax.tick_params(rotation=-35, labelsize = "medium")
            
            for spine in ax.spines.values():
                spine.set_edgecolor('black')
                spine.set_linewidth(1)
                
        def plot_depth_chart(self, depths, y_positions, ax):
            ax.hlines(y=y_positions, xmin=0, xmax=0.2, linestyles='-', linewidth=0.8)
            ax.set_yticks(y_positions)
            ax.set_yticklabels(depths)
            ax.invert_yaxis()
            ax.set_xticklabels([])
            ax.set_title('Depth')
            ax.axis('on')
            ax.set_ylabel(self.drill_hole)
            ax.tick_params(labelsize = "small")
    
        def update_plots(self):
            self.drill_hole = self.drill_hole_combo.get()
    
            filtered_df_0 = self.df_0[self.df_0['Drill_Hole'] == self.drill_hole]
            if filtered_df_0.empty:
                self.output_text.insert("end", "No samples found for the selected drill hole.")
                return
    
            filtered_df_0 = filtered_df_0.sort_values(by='Depth')
    
            sample_indices = filtered_df_0.index
            depths = filtered_df_0['Depth'].tolist()
    
            if len(sample_indices) == 0:
                self.output_text.insert("end", "No samples found for the selected drill hole.")
                return
    
            # Calculate y-positions based on depths
            max_depth = max(depths)
            min_depth = min(depths)
            y_positions = [((depth - min_depth) / (max_depth - min_depth)) * len(depths) for depth in depths]
    
            plt.close('all')
            for widget in self.bargraph_frame.winfo_children():
                widget.destroy()
    
            fig, axes = plt.subplots(nrows=1, ncols=len(self.pca_df_scaled.columns) + 1, figsize=(15, 10))
            
            pos1 = axes[0].get_position()
            axes[0].set_position([pos1.x0, pos1.y0, pos1.width * 0.5, pos1.height])
            
            self.plot_depth_chart(depths, y_positions, axes[0])
    
            for col, pc in enumerate(self.pca_df_scaled.columns):
                data = self.pca_df_scaled.loc[sample_indices, pc]
                ax = axes[col + 1]
                self.plot_pca_scatter(data, y_positions, ax)
                ax.set_title(f'PC{col + 1}')
    
    
            canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
            canvas.draw()
    
            toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            canvas.get_tk_widget().pack(side=ctk.TOP, fill=ctk.BOTH, expand=True)
    except:
        pass






class PCA_class:
    def __init__(self, df, scaler_combo):
        self.df = df
        self.scaler_combo = scaler_combo
        self.perform_pca()
        self.bargraph_frame = None
        
    def perform_pca(self):
        if self.scaler_combo.get() == "Standard Scaler":
            scaling=StandardScaler()
        elif self.scaler_combo.get() == "Logarithmic Scaler":
            scaling = FunctionTransformer(np.log10, validate=True)
        try:
            scaling.fit(self.df)
            Scaled_data = scaling.transform(self.df)
            Scaled_df = pd.DataFrame(Scaled_data)
            
            self.pca = PCA(n_components=6)
            self.pca.fit(Scaled_data)
            self.x = self.pca.transform(Scaled_data)
            self.create_pca_df()
             
        except Exception as e:
            raise ValueError('Not enough components for PCA. Please select more!') from e

    def create_pca_df(self):  
        self.pca_df = pd.DataFrame(
            data=self.x, 
            columns=['PC'+str(i) for i in range(1, len(self.pca.components_)+1)]
        )
        self.scale_pca()
        
    def scale_pca(self):
        self.pca_df_scaled = self.pca_df.copy()
        scaler_df = self.pca_df[self.pca_df.columns]
        scaler = 1 / (scaler_df.max() - scaler_df.min())
        for index in scaler.index:
            self.pca_df_scaled[index] *= scaler[index]






class loading_class:
    def __init__(self, pca_instance, df, bargraph_frame, box_frame, box_frame_sub, on_button_click, apply_button):
        self.apply_button = apply_button
        self.on_button_click = on_button_click
        self.pca = pca_instance.pca
        self.df = df
        self.loadings = pd.DataFrame(self.pca.components_.T, columns=['PC'+str(i) for i in range(1, len(self.pca.components_)+1)], index=df.columns)
        self.loadings.index = self.loadings.index.str.replace('_ppm', '').str.replace('_pct', '').str.replace('(int)', '').str.replace('_Howell', '')
        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.var = IntVar()
        self.group_list = []
        self.Graph_PC()
        #self.var.trace_add("write", self.check)

    def Graph_PC(self):
        self.graph_bar_button = ctk.CTkButton(self.box_frame, text="Graph PC bar graph", command=lambda: self.on_button_click(self.graph_bar_button, self.Graph_PC_sub))
        self.graph_bar_button.pack(side="top", padx=5, pady=(5,0))

    def Graph_PC_sub(self):
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Sort Loadings", variable=self.var)
        self.checkbox.pack(side="top", anchor="w", padx=5, pady=3)

        self.elements_listbox = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        for element in self.loadings.index:
            self.elements_listbox.insert(tk.END, element)
        self.elements_listbox.pack(side="top", padx=5, pady=5)

        self.add_group_button = ctk.CTkButton(self.box_frame_sub, text="Add Group", command=self.add_group)
        self.add_group_button.pack(side="top", padx=5, pady=5)

        self.group_listbox = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        self.group_listbox.pack(side="top", padx=5, pady=5)

        self.apply_Graph_PC = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.update_plots)
        self.apply_Graph_PC.pack(side="top", padx=5, pady=5)

        self.clear_groups_button = ctk.CTkButton(self.box_frame_sub, text="Clear Groups", command=self.clear_groups)
        self.clear_groups_button.pack(side="top", padx=5, pady=5)

    def clear_groups(self):
        self.group_list.clear()
        self.group_listbox.delete(0, tk.END)

    def add_group(self):
        selected_elements = [self.elements_listbox.get(i) for i in self.elements_listbox.curselection()]
        self.group_list.append(selected_elements)
        self.group_listbox.insert(tk.END, f"Group {len(self.group_list)}: {', '.join(selected_elements)}")

    def normalize(self, values, vmin=None, vmax=None):
        vmin = vmin if vmin is not None else np.min(values)
        vmax = vmax if vmax is not None else np.max(values)
        norm_values = (values - vmin) / (vmax - vmin)
        return norm_values

    def plot_pc_barchart(self, data, pc_name, ax, sort, group_indices):
        if sort:
            data = data.sort_values(ascending=False)
        norm_values = self.normalize(data, vmin=-max(abs(data)), vmax=max(abs(data)))
        bar_colors = plt.cm.coolwarm(norm_values)
        data.plot(kind='barh', color=bar_colors, ax=ax, fontsize="x-small")
        ax.set_title(pc_name)
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.set_ylabel(' ')

        # Draw horizontal lines to separate groups
        for idx in group_indices:
            ax.axhline(y=idx - 0.5, color='black', linewidth=1)

    def update_plots(self):
        sort = self.var.get() == 1
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()

        fig, axes = plt.subplots(ncols=len(self.loadings.columns), figsize=(15, 8))
        if len(self.loadings.columns) == 1:
            axes = [axes]

        for i in range(len(self.loadings.columns)):
            combined_data = pd.Series(dtype='float64')
            group_indices = []
            current_index = 0
            selected_indices = self.group_listbox.curselection()
            for index in selected_indices:
                group = self.group_list[index]
                group_data = self.loadings.loc[group, f'PC{i+1}']
                if sort:
                    group_data = group_data.sort_values(ascending=False)
                combined_data = pd.concat([combined_data, group_data])
                current_index += len(group_data)
                group_indices.append(current_index)
            
            if not combined_data.empty:
                self.plot_pc_barchart(combined_data, f'PC{i+1}', axes[i], sort=False, group_indices=group_indices)

        plt.tight_layout()

        fig_c = plt.figure(figsize=(12, 1))
        min_val = self.loadings.min().min()
        max_val = self.loadings.max().max()
        norm = plt.Normalize(vmin=min_val, vmax=max_val)
        cmap = plt.cm.coolwarm
        cbar = fig_c.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes, orientation='horizontal', fraction=0.02)

        canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
        canvas.draw()

        toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=ctk.TOP, fill=ctk.BOTH, expand=True)






class sample_class:
    def __init__(self, pca_instance, df, cleaned_df, bargraph_frame, box_frame, box_frame_sub, on_button_click, apply_button):
        self.apply_button = apply_button
        self.on_button_click = on_button_click
        self.pca = pca_instance.pca
        self.pca_df_scaled = pca_instance.pca_df_scaled
        self.df = df
        self.cleaned_df = cleaned_df
        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.var = IntVar()
        self.Graph_PCA()

    def Graph_PCA(self):
        self.graph_bar_button = ctk.CTkButton(self.box_frame, text="Sample PC bargraph", command=lambda: self.on_button_click(self.graph_bar_button, self.Graph_PCA_sub))
        self.graph_bar_button.pack(side="top", padx=5, pady=(15,0))

    def Graph_PCA_sub(self):
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Sort Loadings", variable=self.var)
        self.checkbox.pack(side="top", anchor="w", padx=5, pady=3)

        self.apply_Graph_PCA = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.update_plots)
        self.apply_Graph_PCA.pack(side="top", padx=5, pady=5)

    def normalize(self, values, vmin=None, vmax=None):
        vmin = vmin if vmin is not None else np.min(values)
        vmax = vmax if vmax is not None else np.max(values)
        norm_values = (values - vmin) / (vmax - vmin)
        return norm_values

    def plot_pca_barchart(self, data, entry_name, ax, sort):
        if sort:
            data = data.sort_values(ascending=False)
        norm_values = self.normalize(data, vmin=-max(abs(data)), vmax=max(abs(data)))
        bar_colors = plt.cm.coolwarm(norm_values)
        data.plot(kind='barh', color=bar_colors, ax=ax, fontsize="xx-small")
        ax.set_title(entry_name)
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.set_ylabel('')

    def update_plots(self):
        sort = self.var.get() == 1
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()

        num_samples = len(self.pca_df_scaled)
        num_pcs = len(self.pca_df_scaled.columns)
        fig, axes = plt.subplots(ncols=num_pcs, figsize=(12, 10))

        for i in range(num_pcs):
            pc_name = f'PC{i+1}'
            data = self.pca_df_scaled.iloc[:, i]
            self.plot_pca_barchart(data, pc_name, axes[i], sort)
        
        for ax in axes:
            ax.set_yticks(range(num_samples))
            ax.set_yticklabels(self.cleaned_df.iloc[:, 0], fontsize="xx-small")

        plt.tight_layout()

        min_val = self.pca_df_scaled.min().min()
        max_val = self.pca_df_scaled.max().max()
        norm = plt.Normalize(vmin=min_val, vmax=max_val)
        cmap = plt.cm.coolwarm

        canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
        canvas.draw()

        toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=ctk.TOP, fill=ctk.BOTH, expand=True)






class threed_class:
    def __init__(self, pca_instance, df, bargraph_frame, box_frame, box_frame_sub, dc, ds, color_map, color_map1, on_button_click, apply_button, legend_frame):
        self.apply_button = apply_button
        self.on_button_click = on_button_click
        self.pca = pca_instance.pca
        self.pca_df_scaled = pca_instance.pca_df_scaled
        self.df = df
        self.dc = dc
        self.ds = ds
        self.color_map = color_map
        self.color_map1 = color_map1

        self.loadings = pd.DataFrame(self.pca.components_.T, columns=['PC'+str(i) for i in range(1, len(self.pca.components_)+1)], index=df.columns)
        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame

        self.var = IntVar()
        self.var1 = IntVar()
        
        try:
            dc.reset_index(drop=True, inplace=True)
            ds.reset_index(drop=True, inplace=True)
        except:
            pass
            
        self.plot_3d()

    def plot_3d(self):
        self.plot_3d_button = ctk.CTkButton(self.box_frame, text="Graph 3D Biplot", command=lambda: self.on_button_click(self.plot_3d_button, self.plot_3d_sub))
        self.plot_3d_button.pack(side="top", padx=5, pady=(5,0))

    def plot_3d_sub(self):
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        pc_options = [f'PC{i+1}' for i in range(len(self.pca.components_))]

        ttk.Label(self.box_frame_sub, text="Select X-Axis:").pack(side="top", padx=5, pady=(5,0))
        self.pc1_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc1_combo.current(0)
        self.pc1_combo.pack(side="top", padx=5, pady=(5,0))

        ttk.Label(self.box_frame_sub, text="Select Y-Axis:").pack(side="top", padx=5, pady=(5,0))
        self.pc2_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc2_combo.current(1)
        self.pc2_combo.pack(side="top", padx=5, pady=(5,0))

        ttk.Label(self.box_frame_sub, text="Select Z-Axis:").pack(side="top", padx=5, pady=(5,0))
        self.pc3_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc3_combo.current(2)
        self.pc3_combo.pack(side="top", padx=5, pady=(5,0))
        
        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.pack(side="top", anchor="w", padx=5, pady=(3,0))

        self.checkbox1 = ctk.CTkCheckBox(self.box_frame_sub, text="Display Legend", variable=self.var1)
        self.checkbox1.pack(side="top", anchor="w", padx=5, pady=(3,0))

        self.multiselect_3d = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        self.multiselect_3d.pack(side="top", padx=5, pady=5)
        for names in self.df.columns.tolist():
            self.multiselect_3d.insert(tk.END, names)

        self.apply_plot_3d = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.show_shape)
        self.apply_plot_3d.pack(side="top", padx=5, pady=5)

    def show_shape(self):
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

        pc1 = self.pc1_combo.get()
        pc2 = self.pc2_combo.get()
        pc3 = self.pc3_combo.get()

        self.fig = plt.figure(figsize=(11, 11))
        self.ax = self.fig.add_subplot(111, projection='3d')
        xdata = self.pca_df_scaled[pc1]
        ydata = self.pca_df_scaled[pc2]
        zdata = self.pca_df_scaled[pc3]

        canvas = FigureCanvasTkAgg(self.fig, master=self.bargraph_frame)
        canvas.draw()

        toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=ctk.RIGHT, fill=ctk.Y)

        try:     
            self.shapes1 = []
            for i in range(len(self.dc)):
                shape1 = self.ax.scatter3D(xdata[i], ydata[i], zdata[i], c=self.dc['Color'][i], marker=self.ds["Shapes"][i])
                self.shapes1.append(shape1)
    
            self.dots = self.ax.scatter3D(xdata, ydata, zdata, c=self.dc['Color'])

        except:
            self.dots = self.ax.scatter3D(xdata, ydata, zdata)


        self.setups(pc1, pc2, pc3)
        self.shape()
        self.legend()

    def shape(self):
        try:
            shape = self.var.get() == 1
            if shape:
                for shape1 in self.shapes1:
                    shape1.set_visible(True)
                self.dots.set_visible(False)
            else:
                self.dots.set_visible(True)
                for shape1 in self.shapes1:
                    shape1.set_visible(False)
        except:
            self.dots.set_visible(True)
            
    def legend(self):
        if not self.ds.empty and not self.ds.empty:
            handle1 = []
            label1 = []
            for lithology, color in self.color_map.items():
                l1 = self.ax.scatter([], [], c=color, label=lithology)
                handle1.append(l1)
                label1.append(lithology)
    
            handle2 = []
            label2 = []
            for shape, shapes in self.color_map1.items():
                l2 = self.ax.scatter([], [], c="black", marker=shapes, label=shape)
                handle2.append(l2)
                label2.append(shape)
    
            total_entries = len(label1) + len(label2)
            height_per_entry = 0.225
            fig_height = total_entries * height_per_entry
    
            figx = plt.figure(figsize=(2.5, fig_height))
            axx = figx.add_subplot(111)
            axx.axis('off')
    
            self.legend1 = plt.legend(handle1, label1, bbox_to_anchor=(0.9, 1.1), title='Lithology', fontsize='small')
            self.legend2 = plt.legend(handle2, label2, bbox_to_anchor=(0.9, 0.3), title='Rock Type', fontsize='small')
            axx.add_artist(self.legend1)
            axx.add_artist(self.legend2)
            self.canvas1 = FigureCanvasTkAgg(figx, master=self.legend_frame)
            self.canvas1.get_tk_widget().pack(fill="both", expand=True)
            
            # legend1 = self.ax.legend(handles=handle1, labels=label1, loc='upper left', title='Lithology')
            # legend2 = self.ax.legend(handles=handle2, labels=label2, loc='lower left', title='Shapes')
    
            # legend1.set_draggable(True)
    
            # self.ax.add_artist(legend1)
            # self.ax.add_artist(legend2)
    
            # canvas = FigureCanvasTkAgg(self.fig, master=self.bargraph_frame)
            # canvas.get_tk_widget().pack(fill="both", expand=True)
            
            self.legend1.set_visible(False)
            self.legend2.set_visible(False)
    
            Legend = self.var1.get() == 1
            if Legend:
                self.legend1.set_visible(True)
                self.legend2.set_visible(True)
            else:
                self.legend1.set_visible(False)
                self.legend2.set_visible(False)
        else:
            pass
    

    def setups(self, pc1, pc2, pc3):
        show_names = self.df.columns.tolist()
        indx = self.df.columns.get_indexer(show_names)
        scale = 2

        loadings = self.pca.components_
        pc1_index = int(pc1[2:]) - 1
        pc2_index = int(pc2[2:]) - 1
        pc3_index = int(pc3[2:]) - 1
        xs = scale * loadings[pc1_index]
        ys = scale * loadings[pc2_index]
        zs = scale * loadings[pc3_index]

        plt.title(f'3D Biplot')

        self.ax.set_xlabel(pc1)
        self.ax.set_ylabel(pc2)
        self.ax.set_zlabel(pc3)
        x_arr = np.zeros(len(loadings[0]))
        y_arr = z_arr = x_arr

        all_points = np.concatenate((xs, ys, zs))
        min_value = np.min(all_points)
        max_value = np.max(all_points)
        axis_range = [min_value, max_value]

        self.ax.plot([x_arr.any() + axis_range[0], x_arr.any() + axis_range[1]], [y_arr.any(), y_arr.any()], [z_arr.any(), z_arr.any()], color='k', linestyle='--')
        self.ax.plot([x_arr.any(), x_arr.any()], [y_arr.any() + axis_range[0], y_arr.any() + axis_range[1]], [z_arr.any(), z_arr.any()], color='k', linestyle='--')
        self.ax.plot([x_arr.any(), x_arr.any()], [y_arr.any(), y_arr.any()], [z_arr.any() + axis_range[0], z_arr.any() + axis_range[1]], color='k', linestyle='--')

        self.arrow_list = []
        for i, name in enumerate(self.df.columns):
            ip = self.df.columns.get_loc(name)
            arrow = self.ax.quiver(x_arr[ip], y_arr[ip], z_arr[ip], xs[ip], ys[ip], zs[ip], color='r', arrow_length_ratio=0.1, linewidth=0.5)
            self.arrow_list.append(arrow)

        self.namelist = []
        for i, names in enumerate(show_names):
            ip = indx[i]
            n = self.ax.text(xs[ip] + 0.02, ys[ip] + 0.02, zs[ip] + 0.02, names, fontsize='small')
            self.namelist.append(n)

        selected_indices = self.multiselect_3d.curselection()
        self.selected_items = [self.multiselect_3d.get(i) for i in selected_indices]
        self.findname_specific()

    def findname_specific(self):
        options = self.df.columns.tolist()
        for i in range(len(options[:])):
            if options[i] in self.selected_items:
                self.namelist[i].set_visible(True)
                self.arrow_list[i].set_visible(True)
            else:
                self.namelist[i].set_visible(False)
                self.arrow_list[i].set_visible(False)



    


class twod_class:
    def __init__(self, pca_instance, df, cleaned_df, bargraph_frame, box_frame, box_frame_sub, dc, ds, color_map, color_map1, on_button_click, apply_button, legend_frame):
        self.pca = pca_instance.pca
        self.pca_df_scaled = pca_instance.pca_df_scaled
        self.df = df
        self.cleaned_df = cleaned_df
        self.dc = dc
        self.ds = ds
        self.color_map = color_map
        self.color_map1 = color_map1

        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame

        self.var = IntVar()
        self.var1 = IntVar()
        self.on_button_click = on_button_click
        self.apply_button = apply_button
        
        self.plot_2d()

    def plot_2d(self):
        self.plot_2d_button = ctk.CTkButton(self.box_frame, text="Graph 2D Biplot", command=lambda: self.on_button_click(self.plot_2d_button, self.plot_2d_sub))
        self.plot_2d_button.pack(side="top", padx=5, pady=(5,0))

    def plot_2d_sub(self):
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        self.pc1_var = tk.StringVar()
        self.pc2_var = tk.StringVar()

        pc_options = [f'PC{i+1}' for i in range(len(self.pca.components_))]
        
        ctk.CTkLabel(self.box_frame_sub, text="Select PC1:").pack(side="top", padx=5, pady=(5,0))
        self.pc1_combo = ttk.Combobox(self.box_frame_sub, textvariable=self.pc1_var, values=pc_options)
        self.pc1_combo.current(0)
        self.pc1_combo.pack(side="top", padx=5, pady=(5,0))
        
        ctk.CTkLabel(self.box_frame_sub, text="Select PC2:").pack(side="top", padx=5, pady=(5,0))
        self.pc2_combo = ttk.Combobox(self.box_frame_sub, textvariable=self.pc2_var, values=pc_options)
        self.pc2_combo.current(1)
        self.pc2_combo.pack(side="top", padx=5, pady=(5,0))
        
        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.pack(side="top", anchor="w", padx=5, pady=(3,0))
        
        self.checkbox1 = ctk.CTkCheckBox(self.box_frame_sub, text="Display Legend", variable=self.var1)
        self.checkbox1.pack(side="top", anchor="w", padx=5, pady=(3,0))

        self.multiselect_2d = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        self.multiselect_2d.pack(side="top", padx=5, pady=5)
        for names in self.df.columns.tolist():
            self.multiselect_2d.insert(tk.END, names)

        self.apply_plot_2d = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.show_shape)
        self.apply_plot_2d.pack(side="top", padx=5, pady=5)

    def show_shape(self):
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
    
        pc1 = self.pc1_var.get()
        pc2 = self.pc2_var.get()
    
        xdata = self.pca_df_scaled[pc1]
        ydata = self.pca_df_scaled[pc2]
        
        fig = plt.figure(figsize=(10, 10))
        
        if 'Lithology' in self.cleaned_df.columns:
            gs = fig.add_gridspec(5, 5, hspace=0.4, wspace=0.4)
            self.ax1 = fig.add_subplot(gs[1:5, 0:4])
            ax_box_x = fig.add_subplot(gs[0, 0:4], sharex=self.ax1)
            ax_box_y = fig.add_subplot(gs[1:5, 4], sharey=self.ax1)

        else:
            self.ax1 = fig.add_subplot(111)
        
        try:
            self.shapes2 = []
            for i in range(len(self.df)):
                shape2 = self.ax1.scatter(xdata[i], ydata[i], c=self.dc['Color'][i], marker=self.ds["Shapes"][i])
                self.shapes2.append(shape2)
            
            self.dots2 = self.ax1.scatter(xdata, ydata, c=self.dc['Color'])
        except:
            self.dots2 = self.ax1.scatter(xdata, ydata)

        
        if 'Lithology' in self.cleaned_df.columns:
            
            filtered_df = self.cleaned_df.copy()
            filtered_df.reset_index(drop=True, inplace=True)
            
            xdata_filtered = self.pca_df_scaled.loc[filtered_df.index, pc1]
            ydata_filtered = self.pca_df_scaled.loc[filtered_df.index, pc2]
    
            # Map the colors to the filtered data
            filtered_colors = filtered_df['Lithology'].map(self.color_map)
            
            unique_lithologies = filtered_df['Lithology'].nunique()
            palette = [self.color_map[lithology] for lithology in filtered_df['Lithology'].unique()]
            
            sns.boxplot(x=xdata_filtered, y=filtered_df['Lithology'], ax=ax_box_x, palette=palette, orient='h')
            sns.boxplot(y=ydata_filtered, x=filtered_df['Lithology'], ax=ax_box_y, palette=palette, orient='v')
    
            ax_box_x.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
            ax_box_x.tick_params(axis='y', labelsize=8, left=False, right=True, labelleft=False, labelright=True)
    
            ax_box_y.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
            ax_box_y.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    
            ax_box_x.set_ylabel(None)
            ax_box_x.set_xlabel(None)
            
            ax_box_y.set_ylabel(None)
            ax_box_y.set_xlabel(None)

        else:
            pass

        plt.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
        canvas.draw()
        
        toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=ctk.RIGHT, fill=ctk.Y)
        
        self.setups(pc1, pc2)
        self.shape()
        self.legend()



    def shape(self):
        try:
            shape = self.var.get() == 1
            if shape:
                for shape2 in self.shapes2:
                    shape2.set_visible(True)
                self.dots2.set_visible(False)
            else:
                self.dots2.set_visible(True)
                for shape2 in self.shapes2:
                    shape2.set_visible(False)
        except:
            self.dots2.set_visible(True)
            
    def legend(self):    
        if not self.dc.empty and not self.ds.empty:
            handle3 = []
            label3 = []
            for lithology, color in self.color_map.items():
                l3 = self.ax1.scatter([], [], c=color, label=lithology)
                handle3.append(l3)
                label3.append(lithology)
            
            handle4 = []
            label4 = []
            for shape, shapes in self.color_map1.items():
                l4 = self.ax1.scatter([], [], c="black", marker=shapes, label=shape)
                handle4.append(l4)
                label4.append(shape)
            
            total_entries = len(label3) + len(label4)
            height_per_entry = 0.225
            fig_height = total_entries * height_per_entry
                
            figx = plt.figure(figsize=(2.5, fig_height))
            axx = figx.add_subplot(111)
            axx.axis('off')
        
            
            self.legend1 = pyplot.legend(handle3, label3, bbox_to_anchor = (0.9,1.1), title='Lithology', fontsize='small')
            self.legend2 = pyplot.legend(handle4, label4, bbox_to_anchor = (0.9,0.3), title='Rock Type', fontsize='small')
            axx.add_artist(self.legend1)
            axx.add_artist(self.legend2)
            self.canvas1 = FigureCanvasTkAgg(figx, master=self.legend_frame)
            self.canvas1.get_tk_widget().pack(fill="both", expand=True)
    
            self.legend1.set_visible(False)
            self.legend2.set_visible(False)
            self.legenda()
        else:
            pass
            
    def legenda(self):
        Legend = self.var1.get() == 1
        if Legend:
            self.legend1.set_visible(True)
            self.legend2.set_visible(True)
        else:
            self.legend1.set_visible(False)
            self.legend2.set_visible(False)

    def setups(self, pc1, pc2):
        show_names = self.df.columns.tolist()
        indx = self.df.columns.get_indexer(show_names)
        
        loadings = self.pca.components_
        pc1_index = int(pc1[2:]) - 1
        pc2_index = int(pc2[2:]) - 1
        xs = loadings[pc1_index]
        ys = loadings[pc2_index]

        arrow_list_2d = []
        
        x_arr = np.zeros(len(loadings[0]))
        y_arr = x_arr

        plt.axhline(y=y_arr.any(), color='k', linestyle='--')
        plt.axvline(x=x_arr.any(), color='k', linestyle='--')
        
        arrow_scale = 1.2
        
        for i, name in enumerate(self.df.columns):
            ip = self.df.columns.get_loc(name)
            arrow_2d = self.ax1.quiver(x_arr[ip], y_arr[ip], xs[ip], ys[ip], color='r', scale=arrow_scale, width=0.002)
            arrow_list_2d.append(arrow_2d)
        
        namelist_2d = []
        for i, names in enumerate(show_names):
            ip = indx[i]
            n = self.ax1.text((1)*xs[ip], (1)*ys[ip], names, fontsize='small')
            namelist_2d.append(n)

        selected_indices = self.multiselect_2d.curselection()
        self.selected_items = [self.multiselect_2d.get(i) for i in selected_indices]
        self.findname_specific_2d(namelist_2d, arrow_list_2d)
        
        self.ax1.set_xlabel(pc1)
        self.ax1.set_ylabel(pc2)
        self.ax1.set_title(f"{pc1} vs {pc2}")

    def findname_specific_2d(self, namelist_2d, arrow_list_2d):
        options = self.df.columns.tolist()
        for i in range(len(options[:])):
            if options[i] in self.selected_items:
                namelist_2d[i].set_visible(True)
                arrow_list_2d[i].set_visible(True)
            else:
                namelist_2d[i].set_visible(False)
                arrow_list_2d[i].set_visible(False)






class Cluster2DPlotClass:
    def __init__(self, df, df_0, file_path, sheet_name, bargraph_frame, box_frame, box_frame_sub, dc, ds, on_button_click, apply_button, legend_frame):
        self.df = df.dropna()
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.dc = dc
        self.ds = ds
        self.on_button_click = on_button_click
        self.apply_button = apply_button
        self.legend_frame = legend_frame
        self.df_0 = df_0
        self.var = IntVar()
        
        try:
            self.max_Zn = self.df['Zn_ppm'].max()
            self.min_Zn = self.df['Zn_ppm'].min()
        except:
            pass
        
        self.plot_2d_cluster()

    def plot_2d_cluster(self):
        self.plot_2d_cluster_button = ctk.CTkButton(self.box_frame, text="Cluster 2D", command=lambda: self.on_button_click(self.plot_2d_cluster_button, self.plot_2d_cluster_sub))
        self.plot_2d_cluster_button.pack(side="top", padx=5, pady=(5, 0))

    def plot_2d_cluster_sub(self):
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 1:").pack(side="top", padx=5, pady=(5, 0))
        self.axis1_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis1_combo.current(0)
        self.axis1_combo.pack(side="top", padx=5, pady=(5, 0))
        
        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 2:").pack(side="top", padx=5, pady=(5, 0))
        self.axis2_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis2_combo.current(1)
        self.axis2_combo.pack(side="top", padx=5, pady=(5, 0))

        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.pack(side="top", anchor="w", padx=5, pady=(3,0))

        ctk.CTkLabel(self.box_frame_sub, text="Number of Clusters:").pack(side="top", padx=5, pady=(5, 0))
        self.k_slider = tk.Scale(self.box_frame_sub, from_=2, to=10, orient=tk.HORIZONTAL)
        self.k_slider.pack(side="top", padx=5, pady=(5, 0))

        self.apply_plot_2d_cluster = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.show_cluster)
        self.apply_plot_2d_cluster.pack(side="top", padx=5, pady=5)

    def yellowbrick(self):
        from yellowbrick.cluster import KElbowVisualizer
        plt.rcParams.update({'font.size': 10})
        
        fig = plt.figure(figsize=(3, 2))
        ax = fig.add_subplot(111)
        ax.axis('off')
        
        model = KMeans(n_init='auto')
        visualizer = KElbowVisualizer(model, k=(1, 12), timing = False, title = "Distortion Score Elbow").fit(self.X)
        visualizer.finalize()
        
        self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
        self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        plt.close(fig)

    def show_cluster(self):
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
        
        axis1 = self.axis1_combo.get()
        axis2 = self.axis2_combo.get()
        k = self.k_slider.get()

        if axis1 != axis2:
            self.df[axis1] = self.df[axis1].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x).astype(float)
            self.df[axis2] = self.df[axis2].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x).astype(float)
            
            self.X = self.df[[axis1, axis2]].values
            self.yellowbrick()

            pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
            ])
            
            pipe.fit(self.X)
            self.df['cluster'] = pipe.named_steps['model'].labels_
            
            rangex = (self.X[:, 0].max()-self.X[:, 0].min())/10
            rangey = (self.X[:, 1].max()-self.X[:, 1].min())/10
            x_min, x_max = self.X[:, 0].min() - rangex, self.X[:, 0].max() + rangex
            y_min, y_max = self.X[:, 1].min() - rangey, self.X[:, 1].max() + rangey
            print (rangex)
            print (rangey)
            
            stepsize = max([y_max - y_min, x_max - x_min]) / 1000
            print (stepsize)
            xx, yy = np.meshgrid(np.arange(x_min, x_max, stepsize),
                                 np.arange(y_min, y_max, stepsize))
            
            
            Z = pipe.named_steps['model'].predict(pipe.named_steps['scale'].transform(np.c_[xx.ravel(), yy.ravel()]))
            Z = Z.reshape(xx.shape)

            fig, ax = plt.subplots(figsize=(9, 7))
            ax = fig.add_subplot(111)
            ax.contourf(xx, yy, Z, cmap=plt.cm.Pastel1, alpha=0.2)
            
            def map_size(x, new_min, new_max):
                old_min = self.min_Zn
                old_max = self.max_Zn
                return ((np.log(x + 1) - np.log(old_min + 1)) / (np.log(old_max + 1) - np.log(old_min + 1))) * (new_max - new_min) + new_min
            
            self.shapes1 = []
            self.df.reset_index(drop=True, inplace=True)
            if not self.ds.empty and not self.dc.empty:
                sizes = self.df['Zn_ppm'].apply(lambda x: map_size(x, 20, 100))
                for i in range(len(self.df)):
                    shape1 = ax.scatter(self.df[axis1][i], self.df[axis2][i],
                        c=self.dc['Color'][i],
                        marker=self.ds["Shapes"][i],
                        s=sizes[i],
                    )
                    self.shapes1.append(shape1)
            else:
                for i in range(len(self.df)):
                    shape1 = ax.scatter(self.df[axis1][i], self.df[axis2][i])
                
            if not self.dc.empty:
                self.dots = ax.scatter(self.df[axis1], self.df[axis2], c=self.dc['Color'], s=self.df['Zn_ppm'].apply(lambda x: map_size(x, 20, 100)))
            else:
                self.dots = ax.scatter(self.df[axis1], self.df[axis2])
                
            self.df.apply(lambda row: ax.text(row[axis1], row[axis2] + 0.1, self.df_0.iloc[row.name, 0], fontsize=7, ha='center'), axis=1)
            
            scaler = pipe.named_steps['scale']
            centroids = pipe.named_steps['model'].cluster_centers_
            centroids_original = scaler.inverse_transform(centroids)
            
            ax.scatter(centroids_original[:, 0], centroids_original[:, 1], 
                       marker="X", c="red", s=30, label="centroids")
            ax.legend()
            ax.set_title("K-Means Clustering")
            ax.set_xlabel(axis1)
            ax.set_ylabel(axis2)
            
            canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
            canvas.draw()
            
            toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            canvas.get_tk_widget().pack(side=ctk.RIGHT, fill=ctk.Y)
            self.shape()
        else:
            self.output_text.insert("end", 'Select two different elements')

    def shape(self):
        shape = self.var.get() == 1
        if shape:
            for shape1 in self.shapes1:
                shape1.set_visible(True)
            self.dots.set_visible(False)
        else:
            self.dots.set_visible(True)
            for shape1 in self.shapes1:
                shape1.set_visible(False)






class Cluster3DPlotClass:
    def __init__(self, df, df_0, file_path, sheet_name, bargraph_frame, box_frame, box_frame_sub, dc, ds, on_button_click, apply_button, legend_frame):
        self.df = df.dropna()
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame
        self.df_0 = df_0
        self.on_button_click = on_button_click
        self.apply_button = apply_button
        self.plot_3d_cluster()
        self.dc = dc
        self.ds = ds
        self.var = IntVar()

    def plot_3d_cluster(self):
        self.plot_3d_cluster_button = ctk.CTkButton(self.box_frame, text="Cluster 3D", command=lambda: self.on_button_click(self.plot_3d_cluster_button, self.plot_3d_cluster_sub))
        self.plot_3d_cluster_button.pack(side="top", padx=5, pady=(5, 0))

    def plot_3d_cluster_sub(self):
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 1:").pack(side="top", padx=5, pady=(5, 0))
        self.axis1_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis1_combo.current(0)
        self.axis1_combo.pack(side="top", padx=5, pady=(5, 0))
        
        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 2:").pack(side="top", padx=5, pady=(5, 0))
        self.axis2_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis2_combo.current(2)
        self.axis2_combo.pack(side="top", padx=5, pady=(5, 0))

        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 3:").pack(side="top", padx=5, pady=(5, 0))
        self.axis3_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis3_combo.current(4)
        self.axis3_combo.pack(side="top", padx=5, pady=(5, 0))
        
        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.pack(side="top", anchor="w", padx=5, pady=(3,0))

        ctk.CTkLabel(self.box_frame_sub, text="Number of Clusters:").pack(side="top", padx=5, pady=(5, 0))
        self.k_slider = tk.Scale(self.box_frame_sub, from_=2, to=10, orient=tk.HORIZONTAL)
        self.k_slider.pack(side="top", padx=5, pady=(5, 0))

        self.apply_plot_3d_cluster = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.show_cluster)
        self.apply_plot_3d_cluster.pack(side="top", padx=5, pady=5)

    def yellowbrick(self):
        from yellowbrick.cluster import KElbowVisualizer  
        plt.rcParams.update({'font.size': 10})
        
        fig = plt.figure(figsize=(3, 2))
        ax = fig.add_subplot(111)
        ax.axis('off')
        
        model = KMeans(n_init='auto')
        visualizer = KElbowVisualizer(model, k=(1, 12), timing = False, title = "Distortion Score Elbow").fit(self.X)
        
        visualizer.finalize()
        
        self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
        self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        plt.close(fig)
        
    def show_cluster(self):
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
            
        axis1 = self.axis1_combo.get()
        axis2 = self.axis2_combo.get()
        axis3 = self.axis3_combo.get()
        k = self.k_slider.get()

        if axis1 != axis2 and axis1 != axis3 and axis2 != axis3:
            for col in [axis1, axis2, axis3]:
                self.df[col] = self.df[col].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
            self.df = self.df.astype({axis1: 'float', axis2: 'float', axis3: 'float'})

            self.X = self.df[[axis1, axis2, axis3]].values
            
            self.yellowbrick()

            pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
            ])
            pipe.fit(self.X)
            self.df['cluster'] = pipe.named_steps['model'].labels_

            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')

            try:
                self.max_Zn = self.df['Zn_ppm'].max()
                self.min_Zn = self.df['Zn_ppm'].min()
            except:
                pass
                
            def map_size(x, new_min, new_max):
                old_min = self.min_Zn
                old_max = self.max_Zn
                return ((np.log(x + 1) - np.log(old_min + 1)) / (np.log(old_max + 1) - np.log(old_min + 1))) * (new_max - new_min) + new_min
            
            self.shapes1 = []
            self.df.reset_index(drop=True, inplace=True)
            
            try:
                sizes = self.df['Zn_ppm'].apply(lambda x: map_size(x, 20, 100))
                for i in range(len(self.df)):
                    shape1 = ax.scatter(self.df[axis1][i], self.df[axis2][i], self.df[axis3][i],
                        c=self.dc['Color'][i],
                        marker=self.ds["Shapes"][i],
                        s=sizes[i],
                    )
                    self.shapes1.append(shape1)
                    
            except:
                for i in range(len(self.df)):
                    shape1 = ax.scatter(self.df.iloc[i][axis1], self.df.iloc[i][axis2], self.df.iloc[i][axis3],)
                    self.shapes1.append(shape1)
                    
            if not self.dc.empty:
                self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3], c=self.dc['Color'], s=self.df['Zn_ppm'].apply(lambda x: map_size(x, 20, 100)))
            else:
                self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3])
                
            self.df.apply(lambda row: ax.text(row[axis1]+1, row[axis2], row[axis3], self.df_0.iloc[row.name, 0], fontsize=7, ha='center'), axis=1)

            scaler = pipe.named_steps['scale']
            centroids = pipe.named_steps['model'].cluster_centers_
            centroids_original = scaler.inverse_transform(centroids)
            
            ax.scatter(centroids_original[:, 0], centroids_original[:, 1], centroids_original[:, 2], 
                       marker="X", c="red", s=100, label="centroids")
        
            ax.set_xlabel(axis1)
            ax.set_ylabel(axis2)
            ax.set_zlabel(axis3)
            plt.legend()
            plt.title("3D K-Means Clustering")

            canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
            canvas.draw()
            
            toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            canvas.get_tk_widget().pack(side=ctk.RIGHT, fill=ctk.Y)
            self.shape()
        else:
            self.output_text.insert("end", 'Select three different elements')

    def shape(self):
        shape = self.var.get() == 1
        if shape:
            for shape1 in self.shapes1:
                shape1.set_visible(True)
            self.dots.set_visible(False)
        else:
            self.dots.set_visible(True)
            for shape1 in self.shapes1:
                shape1.set_visible(False)





    
if __name__ == "__main__":
    app = main()
    app.mainloop()






2024-06-20 16:42:45.724 python[72077:4976511] +[CATransaction synchronize] called within transaction


In [4]:
#prediction
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline
# try:
#     y = dc['Lithology']
#     y = y.values.ravel()
#     X = pca_df_scaled
#     print("Shape of y:", y.shape)
#     print("Shape of X:", X.shape)
    
#     clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
#     clf.fit(X, y)
    
#     print(clf.predict([[0.3, -0.01, 0.1]]))
# except:
#     print ("need more litho group")

# memory_4 = get_memory_usage()
# print(f"Memory Usage After predict: {memory_4:.2f} MB")

In [5]:
            #3d background shading, use only if enough memory (mac impossible)
            # x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
            # y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
            # z_min, z_max = X[:, 2].min() - 1, X[:, 2].max() + 1
            # xx, yy, zz = np.meshgrid(np.linspace(x_min, x_max, 50),
            #                          np.linspace(y_min, y_max, 50),
            #                          np.linspace(z_min, z_max, 50))
        
            # # Predict cluster labels for each point in the mesh grid
            # grid_points = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]
            # grid_points_scaled = pipe.named_steps['scale'].transform(grid_points)
            # Z = pipe.named_steps['model'].predict(grid_points_scaled)
            # Z = Z.reshape(xx.shape)
        
            # ax.scatter(xx, yy, zz, c=Z, cmap='Pastel1', alpha=0.1)